Importação das Bibliotecas

In [11]:
import os
import zipfile
import shutil
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import glob

Diretório das Imagens e Saídas de .CSV e DataFrame

In [2]:
# Treino
origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/treino/"
diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/treino_processado/"
# Teste
# origem = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste/"
# diretorio_imagens = "/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/"

Extrair as imagens para o diretório "treino_processado" e adicionando um número para diferenciar as imagens de mesmo nome.

In [3]:

numero_imagem = 1

# Verifica se o diretório de destino existe, caso contrário, cria-o
if not os.path.exists(diretorio_imagens):
    os.makedirs(diretorio_imagens)

# Percorre todas as pastas compactadas no diretório de origem
for pasta_compactada in os.listdir(origem):
    caminho_pasta_compactada = os.path.join(origem, pasta_compactada)

    # Verifica se o item é uma pasta compactada
    if zipfile.is_zipfile(caminho_pasta_compactada):
        # Extrai o conteúdo do arquivo compactado para uma pasta temporária
        pasta_temporaria = os.path.join(diretorio_imagens, "temp")
        with zipfile.ZipFile(caminho_pasta_compactada, 'r') as zip_ref:
            zip_ref.extractall(pasta_temporaria)

        # Percorre todas as imagens na pasta temporária
        for nome_imagem in os.listdir(pasta_temporaria):
            caminho_imagem = os.path.join(pasta_temporaria, nome_imagem)

            # Verifica se o item é um arquivo de imagem
            if os.path.isfile(caminho_imagem):
                # Define o novo nome da imagem com o número no início
                novo_nome_imagem = f"{numero_imagem}_{nome_imagem}"

                # Move a imagem para o diretório de destino com o novo nome
                caminho_destino_imagem = os.path.join(diretorio_imagens, novo_nome_imagem)
                shutil.move(caminho_imagem, caminho_destino_imagem)

                # Incrementa o número para a próxima imagem
                numero_imagem += 1

        # Remove a pasta temporária após processar todas as imagens
        shutil.rmtree(pasta_temporaria)

print("Descompactação concluída.")

Descompactação concluída.


Criação das máscaras. 

In [4]:
def modificar_nome_arquivo(nome_arquivo):
    # Remover a sequência "_(Raw)"
    nome_arquivo_modificado = nome_arquivo.replace("_(Raw)", "")
    
    # Substituir "Wildf" por "F"
    nome_arquivo_modificado = nome_arquivo_modificado.replace("Wildf", "F")
    
    return nome_arquivo_modificado


def limiar(diretorio):
    # Percorre todas as imagens no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(".tiff"):
            # Limpar o nome do arquivo
            nome_arquivo_limpo = modificar_nome_arquivo(nome_arquivo)

            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)

            # Aplicar o limiar
            limiar = 180
            _, thresh = cv2.threshold(imagem, limiar, 1, cv2.THRESH_BINARY)

            # Obter o nome do arquivo sem a extensão
            nome_sem_extensao = os.path.splitext(nome_arquivo_limpo)[0]

            # Salvar a imagem limiarizada com "_Mask" antes da extensão no mesmo diretório
            nome_mask = nome_sem_extensao + "_Mask.tiff"
            caminho_mask = os.path.join(diretorio, nome_mask)
            cv2.imwrite(caminho_mask, thresh)

    # Libera a memória das imagens
    cv2.destroyAllWindows()

limiar(diretorio_imagens)


Recorte das imagens em 64x64

In [5]:
def recortar_imagens(diretorio, tamanho_recorte):
    # Percorre todas as imagens no diretório
    for nome_arquivo in os.listdir(diretorio):
        if nome_arquivo.endswith(".tiff"):
            # Carrega a imagem
            caminho_arquivo = os.path.join(diretorio, nome_arquivo)
            imagem = cv2.imread(caminho_arquivo)
            
            # Obtém as dimensões da imagem
            altura, largura, _ = imagem.shape
            
            # Calcula o número de recortes horizontais e verticais
            num_recortes_horizontais = largura // tamanho_recorte
            num_recortes_verticais = altura // tamanho_recorte
            
            # Realiza o recorte e salva as imagens resultantes
            for i in range(num_recortes_horizontais):
                for j in range(num_recortes_verticais):
                    # Define as coordenadas do retângulo de recorte
                    esquerda = i * tamanho_recorte
                    superior = j * tamanho_recorte
                    direita = (i + 1) * tamanho_recorte
                    inferior = (j + 1) * tamanho_recorte
                    
                    # Realiza o recorte
                    recorte = imagem[superior:inferior, esquerda:direita]
                    
                    # Salva o recorte em um novo arquivo
                    nome_recorte = f"recorte_{i}_{j}_{nome_arquivo}"
                    caminho_recorte = os.path.join(diretorio, nome_recorte)
                    cv2.imwrite(caminho_recorte, recorte)
                    
            # Remove a imagem original
            os.remove(caminho_arquivo)
            
    # Libera a memória das imagens
    cv2.destroyAllWindows()

tamanho_recorte = 64
recortar_imagens(diretorio_imagens, tamanho_recorte)


Transformar imagens para .CSV

In [6]:
def tiff_to_csv(image_directory):
    # Listar todos os arquivos no diretório
    file_list = os.listdir(image_directory)

    # Filtrar apenas os arquivos com extensão .tiff
    tiff_files = [file for file in file_list if file.endswith('.tiff')]

    if not tiff_files:
        print("Nenhum arquivo .tiff encontrado no diretório.")
        return

    # Percorrer cada imagem .tiff
    for tiff_file in tiff_files:
        # Obter o caminho completo do arquivo .tiff
        image_path = os.path.join(image_directory, tiff_file)

        # Abrir a imagem .tiff
        image = Image.open(image_path)

        if 'Wildfires' not in tiff_file:
            # Converter a imagem para escala de cinza
            gray_image = image.convert("L")

            # Converter a imagem em escala de cinza para um array NumPy
            gray_array = np.array(gray_image)
            # Aplainar o array para um vetor 1D
            csv_data = gray_array.flatten()
        else:
            csv_data = []
            for y in range(image.height):
                for x in range(image.width):
                    # Extrair os valores de pixel desejados
                    pixel_value = image.getpixel((x, y))
                    csv_data.append(pixel_value)
        

        # Criar um DataFrame a partir dos dados CSV
        df = pd.DataFrame([csv_data])

        # Identificar o tipo de imagem com base no nome do arquivo
        if 'B11_(Raw)' in tiff_file:
            csv_filename = 'banda11.csv'
        elif 'B12_(Raw)' in tiff_file:
            csv_filename = 'banda12.csv'
        elif 'Wildfires' in tiff_file:
            csv_filename = 'wildfires.csv'
        elif 'B11_Mask' in tiff_file:
            csv_filename = 'mask_banda11.csv'
        elif 'B12_Mask' in tiff_file:
            csv_filename = 'mask_banda12.csv'
        elif 'Fires_Mask' in tiff_file:
            csv_filename = 'mask_fires.csv'
        else:
            print(f"Arquivo não correspondente: {tiff_file}")
            continue

        # Caminho completo do arquivo .csv
        csv_path = os.path.join(image_directory, csv_filename)

        # Escrever o DataFrame para o arquivo .csv
        df.to_csv(csv_path, index=False, header=False,mode='a')

        print(f"Conversão concluída para {tiff_file}. Arquivo CSV gerado: {csv_filename}")

tiff_to_csv(diretorio_imagens)


Conversão concluída para recorte_0_3_6_2023-05-28-00:00_2023-05-28-23:59_Sentinel-2_L2A_Wildfires.tiff. Arquivo CSV gerado: wildfires.csv
Conversão concluída para recorte_3_5_4_2023-05-28-00:00_2023-05-28-23:59_Sentinel-2_L2A_B11_(Raw).tiff. Arquivo CSV gerado: banda11.csv
Conversão concluída para recorte_1_0_6_2023-05-28-00:00_2023-05-28-23:59_Sentinel-2_L2A_Fires_Mask.tiff. Arquivo CSV gerado: mask_fires.csv
Conversão concluída para recorte_0_0_6_2023-05-28-00:00_2023-05-28-23:59_Sentinel-2_L2A_Fires_Mask.tiff. Arquivo CSV gerado: mask_fires.csv
Conversão concluída para recorte_0_0_3_2023-06-21-00:00_2023-06-21-23:59_Sentinel-2_L2A_Fires_Mask.tiff. Arquivo CSV gerado: mask_fires.csv
Conversão concluída para recorte_2_2_5_2023-05-28-00:00_2023-05-28-23:59_Sentinel-2_L2A_B12_Mask.tiff. Arquivo CSV gerado: mask_banda12.csv
Conversão concluída para recorte_1_1_6_2023-05-28-00:00_2023-05-28-23:59_Sentinel-2_L2A_Wildfires.tiff. Arquivo CSV gerado: wildfires.csv
Conversão concluída para rec

Criação dos DataFrame

In [7]:
def csv_to_dataframe(diretorio_imagens):
    # Dicionário para armazenar os DataFrames de cada arquivo CSV
    dfs = {}

    # Percorrer todos os arquivos no diretório
    for filename in os.listdir(diretorio_imagens):
        if filename.endswith('.csv'):
            # Obter o caminho completo do arquivo CSV
            filepath = os.path.join(diretorio_imagens, filename)

            # Ler o arquivo CSV e criar um DataFrame
            df = pd.read_csv(filepath)

            # Adicionar o DataFrame ao dicionário usando o nome do arquivo como chave
            dfs[filename] = df

            # Criar o novo nome de arquivo para o DataFrame
            new_filename = f"df_{os.path.splitext(filename)[0]}.csv"

            # Salvar o DataFrame em um arquivo CSV no mesmo diretório com o novo nome
            df.to_csv(os.path.join(diretorio_imagens, new_filename), index=False, header=list(range(1, len(df.columns)+1)))

            # Imprimir o nome do DataFrame gerado
            print(f"DataFrame gerado para o arquivo {new_filename}")

    return dfs

dfs = csv_to_dataframe(diretorio_imagens)


DataFrame gerado para o arquivo df_mask_fires.csv
DataFrame gerado para o arquivo df_mask_banda11.csv
DataFrame gerado para o arquivo df_mask_banda12.csv
DataFrame gerado para o arquivo df_banda11.csv
DataFrame gerado para o arquivo df_wildfires.csv
DataFrame gerado para o arquivo df_banda12.csv


In [15]:
# Padrão para os arquivos CSV
padrao_arquivos = "*.csv"

# Lista de caminhos para os arquivos CSV
caminhos_df = glob.glob(f"{diretorio_imagens}/{padrao_arquivos}")

# Loop para ler e imprimir o cabeçalho de cada DataFrame
for caminho in caminhos_df:
    df = pd.read_csv(caminho)
    print(f"Cabeçalho do DataFrame {caminho}:")
    print(df.shape)
    print(df.head())
    print()  # Linha em branco para separar os resultados


Cabeçalho do DataFrame /home/cristiano/Imagens/imagens_originais/Sentinel-2/teste2img_processado/mask_fires.csv:
(38, 4096)
   0  0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  ...  0.4042  0.4043  \
0  0    0    0    0    0    0    0    0    0    0  ...       0       0   
1  0    0    0    0    0    0    0    0    0    0  ...       0       0   
2  0    0    0    0    0    0    0    0    0    0  ...       0       0   
3  0    0    0    0    0    0    0    0    0    0  ...       0       0   
4  0    0    0    0    0    0    0    0    0    0  ...       0       0   

   0.4044  0.4045  0.4046  0.4047  0.4048  0.4049  0.4050  0.4051  
0       0       0       0       0       0       0       0       0  
1       0       0       0       0       0       0       0       0  
2       0       0       0       0       0       0       0       0  
3       0       0       0       0       0       0       0       0  
4       0       0       0       0       0       0       0       0  

[5 rows x 4096 columns

In [8]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Sentinel-2/treino_processado/df_banda11.csv")
print(train.shape)
train.head()

(38, 4096)


,1,2,3,4,5,6,7,8,9,10,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
0,81,80,79,79,82,87,88,87,84,81,...,84,84,84,82,81,82,83,83,83,88
1,80,73,69,67,66,65,61,55,49,44,...,84,82,82,82,81,78,77,76,77,80
2,89,88,87,86,86,86,86,86,88,89,...,99,98,98,96,96,97,97,97,97,96
3,105,102,98,96,94,93,93,92,92,92,...,97,96,94,92,92,93,95,98,98,96
4,75,77,79,77,76,78,81,80,77,78,...,85,87,88,88,88,89,88,88,91,92


In [9]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/df_wildfires.csv")
print(train.shape)
train.head()

(38, 4096)


,1,2,3,4,5,6,7,8,9,10,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
0,"(90, 77, 52)","(93, 76, 54)","(96, 80, 57)","(94, 78, 57)","(86, 73, 51)","(81, 67, 52)","(88, 75, 54)","(86, 74, 54)","(88, 76, 54)","(91, 77, 56)",...,"(81, 65, 44)","(88, 70, 46)","(87, 74, 51)","(89, 74, 53)","(93, 75, 52)","(85, 73, 51)","(84, 69, 50)","(90, 75, 54)","(95, 78, 56)","(91, 76, 54)"
1,"(110, 86, 59)","(110, 88, 60)","(110, 88, 60)","(106, 85, 58)","(110, 88, 60)","(116, 94, 66)","(114, 92, 64)","(108, 86, 58)","(100, 82, 55)","(102, 82, 55)",...,"(113, 92, 64)","(113, 91, 62)","(102, 83, 57)","(99, 80, 56)","(100, 82, 57)","(105, 87, 60)","(110, 87, 60)","(104, 86, 60)","(101, 82, 56)","(100, 81, 56)"
2,"(67, 67, 45)","(68, 65, 47)","(58, 60, 41)","(57, 58, 42)","(60, 60, 42)","(72, 67, 47)","(64, 63, 45)","(59, 58, 40)","(62, 61, 44)","(78, 69, 51)",...,"(78, 66, 48)","(78, 64, 43)","(84, 69, 46)","(86, 69, 46)","(83, 70, 47)","(82, 67, 47)","(74, 63, 41)","(72, 64, 40)","(87, 70, 46)","(96, 74, 48)"
3,"(99, 75, 52)","(92, 73, 48)","(83, 71, 49)","(90, 73, 49)","(92, 70, 50)","(86, 70, 49)","(93, 74, 50)","(93, 74, 50)","(94, 73, 49)","(89, 73, 49)",...,"(105, 81, 53)","(101, 77, 51)","(99, 77, 51)","(103, 79, 53)","(99, 80, 53)","(102, 79, 53)","(102, 79, 53)","(105, 80, 54)","(102, 80, 54)","(99, 79, 53)"
4,"(109, 82, 55)","(103, 82, 54)","(107, 82, 54)","(108, 85, 57)","(106, 83, 54)","(105, 80, 54)","(115, 88, 58)","(114, 88, 58)","(110, 83, 55)","(105, 82, 55)",...,"(40, 59, 32)","(40, 59, 32)","(39, 60, 34)","(39, 59, 32)","(39, 59, 32)","(39, 59, 34)","(39, 61, 32)","(40, 60, 32)","(41, 61, 33)","(40, 60, 33)"


In [10]:
train = pd.read_csv("/home/cristiano/Imagens/imagens_originais/Sentinel-2/teste_processado/df_mask_banda12.csv")
print(train.shape)
train.head()

(38, 4096)


,1,2,3,4,5,6,7,8,9,10,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,4096
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
